In [364]:
import pyspark 
import json
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read.json("donation_np.json")

In [365]:
df = df.withColumnRenamed("Contribution Mode","mode_of_payment").withColumnRenamed("Financial Year" , "fin_year").withColumnRenamed("PAN Given","pan_given")
df= df.na.drop(subset=["Name"])

In [366]:
from pyspark.sql.functions import *
df = df.withColumn("Address",sha2(concat_ws("||",df.Address),256))

In [367]:
df = df.drop("field9","field10","field11","field12","field13","field14")

In [368]:
df = df.withColumn(
    "mode_of_payment",
    when(col("mode_of_payment").like("%D.D%") , "Others")
    .when(col("mode_of_payment").like("%CASH%"),"CASH")
    .when(col("mode_of_payment").like("%Bank%"),"BANK").otherwise("CHEQUE"))

In [369]:
x=df.withColumn("Amount",df.Amount.cast('int'))

# SUM

BJP_sum = x.where(x.Party=="BJP").select(sum("Amount"))
INC_sum = x.where(df.Party=="INC").select(sum("Amount"))
NCP_sum = x.where(x.Party=="NCP").select(sum("Amount"))
CPI_sum = x.where(x.Party=="CPI").select(sum("Amount"))
CPIM_sum = x.where(x.Party=="CPI(M)").select(sum("Amount"))

all_list = x.groupBy('Party').agg(count('Amount'),avg('Amount'),max('Amount'))

# COUNT

BJP_Donor = all_list.collect()[1][1]
INC_Donor = all_list.collect()[0][1]
NCP_Donor = all_list.collect()[2][1]
CPI_Donor = all_list.collect()[3][1]
CPIM_Donor = all_list.collect()[4][1]

# AVG

BJP_avg = all_list.collect()[1][2]
INC_avg = all_list.collect()[0][2]
NCP_avg = all_list.collect()[2][2]
CPI_avg = all_list.collect()[3][2]
CPIM_avg = all_list.collect()[4][2]

# MAX

BJP_max = all_list.collect()[1][3]
INC_max = all_list.collect()[0][3]
NCP_max = all_list.collect()[2][3]
CPI_max = all_list.collect()[3][3]
CPIM_max = all_list.collect()[4][3]

# TOP DONOR

BJP_top_donor = x.filter((x.Party=="BJP") & (x.Amount == all_list.collect()[0][3])).first()["Name"]
INC_top_donor = x.filter((x.Party=="INC") & (x.Amount == all_list.collect()[1][3])).first()["Name"]
NCP_top_donor = x.filter((x.Party=="NCP") & (x.Amount == all_list.collect()[2][3])).first()["Name"]
CPI_top_donor = x.filter((x.Party=="CPI") & (x.Amount == all_list.collect()[3][3])).first()["Name"]
CPIM_top_donor = x.filter((x.Party=="CPI(M)") & (x.Amount == all_list.collect()[4][3])).first()["Name"]


print(NCP_top_donor)

#x.groupBy('mode_of_payment').agg(count('mode_of_payment')).show()




#df.select('fin_year').distinct().show()

Lodha Dwellers Pvt. Ltd.


In [370]:

y = x.withColumn('INC_SUM_LTD',when(x.Party=='INC',INC_sum.first()["sum(Amount)"]).otherwise(0))
y = y.withColumn('BJP_SUM_LTD',when(x.Party=='BJP',BJP_sum.first()["sum(Amount)"]).otherwise(0))
y = y.withColumn('NCP_SUM_LTD',when(x.Party=='NCP',NCP_sum.first()["sum(Amount)"]).otherwise(0))
y = y.withColumn('CPI_SUM_LTD',when(x.Party=='CPI',CPI_sum.first()["sum(Amount)"]).otherwise(0))
y = y.withColumn('CPIM_SUM_LTD',when(x.Party=='CPI(M)',CPIM_sum.first()["sum(Amount)"]).otherwise(0))




In [371]:
z = y.withColumn('INC_COUNT_LTD',when(x.Party=='INC',INC_Donor).otherwise(0))
z = z.withColumn('BJP_COUNT_LTD',when(x.Party=='BJP',BJP_Donor).otherwise(0))
z = z.withColumn('NCP_COUNT_LTD',when(x.Party=='NCP',NCP_Donor).otherwise(0))
z = z.withColumn('CPI_COUNT_LTD',when(x.Party=='CPI',CPI_Donor).otherwise(0))
z = z.withColumn('CPIM_COUNT_LTD',when(x.Party=='CPI(M)',CPIM_Donor).otherwise(0))


In [372]:
z = z.withColumn('INC_AVG_LTD',when(x.Party=='INC',INC_avg).otherwise(0))
z = z.withColumn('BJP_AVG_LTD',when(x.Party=='BJP',BJP_avg).otherwise(0))
z = z.withColumn('NCP_AVG_LTD',when(x.Party=='NCP',NCP_avg).otherwise(0))
z = z.withColumn('CPI_AVG_LTD',when(x.Party=='CPI',CPI_avg).otherwise(0))
z = z.withColumn('CPIM_AVG_LTD',when(x.Party=='CPI(M)',CPIM_avg).otherwise(0))


In [373]:
z = z.withColumn('INC_MAX_LTD',when(x.Party=='INC',INC_max).otherwise(0))
z = z.withColumn('BJP_MAX_LTD',when(x.Party=='BJP',BJP_max).otherwise(0))
z = z.withColumn('NCP_MAX_LTD',when(x.Party=='NCP',NCP_max).otherwise(0))
z = z.withColumn('CPI_MAX_LTD',when(x.Party=='CPI',CPI_max).otherwise(0))
z = z.withColumn('CPIM_MAX_LTD',when(x.Party=='CPI(M)',CPIM_max).otherwise(0))


In [374]:
z = z.withColumn('INC_TOP_DONOR',when(x.Party=='INC',INC_top_donor).otherwise(""))
z = z.withColumn('BJP_TOP_DONOR',when(x.Party=='BJP',BJP_top_donor).otherwise(""))
z = z.withColumn('NCP_TOP_DONOR',when(x.Party=='NCP',NCP_top_donor).otherwise(""))
z = z.withColumn('CPI_TOP_DONOR',when(x.Party=='CPI',CPI_top_donor).otherwise(""))
z = z.withColumn('CPIM_TOP_DONOR',when(x.Party=='CPI(M)',CPIM_top_donor).otherwise(""))


In [375]:
INC_years = x.groupBy('Party','fin_year').sum('Amount').filter(x.Party == 'INC').sort("fin_year")
BJP_years = x.groupBy('Party','fin_year').sum('Amount').filter(x.Party == 'BJP').sort("fin_year")
NCP_years = x.groupBy('Party','fin_year').sum('Amount').filter(x.Party == 'NCP').sort("fin_year")
CPI_years = x.groupBy('Party','fin_year').sum('Amount').filter(x.Party == 'CPI').sort("fin_year")
CPIM_years = x.groupBy('Party','fin_year').sum('Amount').filter(x.Party == 'CPI(M)').sort("fin_year")



In [376]:


k = x.groupBy('mode_of_payment').agg(count('mode_of_payment'))

z = z.withColumn('Count_of_donation_by_cheque',when(x.mode_of_payment=='CHEQUE',k.collect()[0][1]).otherwise(""))
z = z.withColumn('Count_of_donation_by_bank',when(x.mode_of_payment=='BANK',k.collect()[1][1]).otherwise(""))
z = z.withColumn('Count_of_donation_by_cash',when(x.mode_of_payment=='CASH',k.collect()[2][1]).otherwise(""))
z = z.withColumn('Count_of_donation_by_others',when(x.mode_of_payment=='Others',k.collect()[3][1]).otherwise(""))



In [ ]:

for i in range(12):    
    name = 'SUM_of_donation_by_year_in_by_INC' + INC_years.collect()[i][1]
    z=z.withColumn(name,when( (x.Party=='INC') & (x.fin_year==INC_years.collect()[i][1]),INC_years.collect()[i][2]).otherwise(0))

for i in range(12):    
    name = 'SUM_of_donation_by_year_in_by_BJP' + BJP_years.collect()[i][1]
    z=z.withColumn(name,when( (x.Party=='BJP') & (x.fin_year==BJP_years.collect()[i][1]),BJP_years.collect()[i][2]).otherwise(0))

for i in range(8):    
    name = 'SUM_of_donation_by_year_in_by_NCP' + NCP_years.collect()[i][1]
    z=z.withColumn(name,when( (x.Party=='NCP') & (x.fin_year==NCP_years.collect()[i][1]),NCP_years.collect()[i][2]).otherwise(0))
    
for i in range(12):    
    name = 'SUM_of_donation_by_year_in_by_CPI' + CPI_years.collect()[i][1]
    z=z.withColumn(name,when( (x.Party=='CPI') & (x.fin_year==CPI_years.collect()[i][1]),CPI_years.collect()[i][2]).otherwise(0))

for i in range(12):    
    name = 'SUM_of_donation_by_year_in_by_CPIM' + CPIM_years.collect()[i][1]
    z=z.withColumn(name,when( (x.Party=='CPIM') & (x.fin_year==CPIM_years.collect()[i][1]),CPIM_years.collect()[i][2]).otherwise(0)) 



  

In [ ]:
z.write.parquet("final_donation.parquet")